In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://marriott-hotels.marriott.com/locations/"
driver.get(url)
time.sleep(30)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'swell_vertical-xs'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
hotelsurl_df = pd.DataFrame(hotel_list)
hotelsurl_df

,name,url
0,Algiers Marriott Hotel Bab Ezzouar,https://www.marriott.com/ALGMC
1,Constantine Marriott Hotel,https://www.marriott.com/CZLMC
2,Cairo Marriott Hotel & Omar Khayyam Casino,https://www.marriott.com/CAIEG
3,"Marriott Mena House, Cairo",https://www.marriott.com/CAIMN
4,Hurghada Marriott Beach Resort,https://www.marriott.com/HRGEG
...,...,...
659,Bogota Marriott Hotel,https://www.marriott.com/BOGMC
660,Cali Marriott Hotel,https://www.marriott.com/CLOMC
661,Medellin Marriott Hotel,https://www.marriott.com/MDEMC
662,Santa Marta Marriott Resort Playa Dormida,https://www.marriott.com/SMRMC


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in hotelsurl_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Marriott'
        except:
            hotel['brand'] = 'Marriott'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df     

{'brand': 'Marriott', 'name': 'Algiers Marriott Hotel Bab Ezzouar', 'url': 'https://www.marriott.com/ALGMC', 'street': 'Trust Complex Buildings, Nouveau Quartier des Affaires, Bab Ezzouar', 'locality': 'Algiers', 'state': '', 'postalcode': '16311', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Discover the Marriott hotel, located in the heart of Babezzouar, at 5 minutes from the airport, 20 minutes from the city center and 10 minutes from the exhibition center (SAFEX Pins Maritimes), it is the essence of refinement. The Marriott combine modernity and simplicity to offer its customers a luxurious hotel. The Marriott hotel has 213 fully equipped rooms to respond to all your needs, offering quality accommodation and 24-hour "At your service" service for all requests. Equipped with the entire necessary infrastructure, the Marriott Hotel will meet both the expectations of customers traveling for business, using its 06 conference ro

{'brand': 'Marriott', 'name': 'Accra Marriott Hotel', 'url': 'https://www.marriott.com/ACCMC', 'street': 'Liberation Road, Airport City', 'locality': 'Accra', 'state': '', 'postalcode': '', 'coordinate1': '-0.176706', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Style and substance blend seamlessly in Ghana at Accra Marriott Hotel. Located near Accra Mall and the beach, our family-friendly hotel is less than 1 kilometer from Kotoka International Airport, and we offer a complimentary airport shuttle, making travel hassle-free. Plus, the city center is a short drive away. Settle into one of our rooms or suites, which feature contemporary design, complimentary Wi-Fi, marble bathrooms, oversized windows and room service 24 hours a day. Visit the hotel fitness center, or take a cooling dip in the pool. Delectable dining options include Red Red, which offers buffet and à la carte options, as well as a lively bar and poolside restaurant. If you\'re plann

{'brand': 'Marriott', 'name': 'Sousse Pearl Marriott Resort & Spa', 'url': 'https://www.marriott.com/MIRMS', 'street': 'Abdelhamid El Kadhi Avenue', 'locality': 'Sousse', 'state': '', 'postalcode': '4000', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ The Pearl Resort &amp; Spa Sousse is a place that defines trends with all of its 222 guest rooms and suites which are newly renovated and beautifully designed providing stunning views of either the city or the sea. All rooms are equipped with modern amenities: Free highspeed, wireless Internet, satellite TV, mini-bar, tea and coffee making facilities, safety-box, hairdryer and ironing board. Our Accessible rooms have easy access for wheelchairs and are spacious for easy mobility.The Resort offers a wide selection of restaurants and bars to enjoy international dishes in our all-day dining restaurant or Asian dishes at the “Opus”.Enjoy cocktails and tasty light snacks in one of our

{'brand': 'Marriott', 'name': 'Beijing Marriott Hotel Northeast', 'url': 'https://www.marriott.com/BJSES', 'street': '26A Xiao Yun Road, Chaoyang District', 'locality': 'Beijing', 'state': '', 'postalcode': '100125', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/bjses-beijing-marriott-hotel-northeast/?directPageRequest=true', 'coordinate2': '39.959861', 'coordinate3': '116.4673626', 'coordinate4': 'N/A', 'description': "[ Beijing Marriott Hotel Northeast is an international business hotel ideally situated in Beijing's Chaoyang CBD, in close proximity to the city's Third Embassy District. Our hotel in downtown Beijing is close to Metro Line 10, Airport Express Sanyuanqiao Station and Beijing Capital International Airport for convenient access to transportation. Featuring spectacular views, our Beijing hotel offers deluxe guest rooms and suites meeting international standards of comfort and technology, including 42-inch LCD TVs, high-speed Internet access and premium amenit

{'brand': 'Marriott', 'name': 'Fuzhou Marriott Hotel Riverside', 'url': 'https://www.marriott.com/FOCMC', 'street': 'No.18 Hujiang Road, Fujian Digital Industrial Park, Changle District', 'locality': 'Fuzhou', 'state': '', 'postalcode': '350207', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Fuzhou Marriott Hotel Riverside is situated inside the Donghu Digital Town, Changle district of Fuzhou City. Hotel is only 5 minutes’ walk to the Fuzhou Digital China Convention &amp; Exhibition Center, 15 minutes’ drive to Changle International Airport. Hotel has a total of 319 elegant rooms and suites. All rooms allows you to either overlook the magnificent East China Sea or the beautiful view of East Lake. Hotel has 3 restaurants and 1 lobby lounge where guest could experience modern and creative Asia Cuisine, classic Western dishes as well as seasonal buffet. Hotel possesses a well-equipped fitness center and an indoor swimming pool to

...

{'brand': 'Marriott', 'name': "Marriott's Manor Club at Ford's Colony", 'url': 'https://www.marriott.com/PHFVI', 'street': '101 St. Andrews Drive', 'locality': 'Williamsburg', 'state': 'Virginia', 'postalcode': '23188', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Create lasting memories of a vacation life well lived by choosing Marriott's Manor Club at Ford's Colony. Ideal for families or treating yourself to a relaxing getaway; you can have it all at our premium vacation ownership resort, located in historic Williamsburg, Virginia. Spacious resort villas are set within an award-winning private community; our resort also offers a pool, sauna, fitness center and a host of outdoor activities like tennis, biking and three championship golf courses with 54-holes. Stylish vacation villas include free Wi-Fi, plush bedding and sofa beds; our condo-style resort rentals boast full kitchens, washer/dryers and furnished balconies. Dine

{'brand': 'Marriott', 'name': 'Marriott Tacoma Downtown', 'url': 'https://www.marriott.com/SEAOM', 'street': '1538 Commerce Street', 'locality': 'Tacoma', 'state': 'Washington', 'postalcode': '98402', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Elevate your stay at the all-new Marriott Tacoma Downtown. Our four-star hotel in Downtown Tacoma, WA, offers 22 stories of stylish décor, comfortable accommodations and modern amenities. From the moment you walk through the grand entrance, our personable staff is here to make sure you get everything you need for an exceptional travel experience. Rest well with plush Marriott bedding in all our rooms and suites. Book select accommodations to enjoy breathtaking views of the Tacoma Waterfront and Mount Rainier. For exclusive perks in our M Club lounge, upgrade to our concierge-level rooms. Maintain your momentum at our onsite fitness center offering digital on-demand fitness classes. He

{'brand': 'Marriott', 'name': 'Milwaukee Marriott West', 'url': 'https://www.marriott.com/MKEMW', 'street': 'W231N1600 Corporate Ct', 'locality': 'Waukesha', 'state': 'Wisconsin', 'postalcode': '53186', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/mkemw-milwaukee-marriott-west/?directPageRequest=true', 'coordinate3': '43.051789', 'coordinate4': '-88.207365', 'description': "[ Explore Milwaukee with the help of Milwaukee Marriott West. Our hotel has 281 deluxe guestrooms with generous amenities, including a 24-hour business center, indoor pool and whirlpool, fitness center, concierge lounge and complimentary parking. Savor breakfast, lunch or dinner at our BLVD Kitchen &amp; Bar® which is an exciting new American bistro that adds an unexpected twist to traditional bar and grill fare. Business travelers, groups and event planners benefit from our hotel's convenient location in Waukesha and access to Interstate 94, providing access to downtown and 

{'brand': 'Marriott', 'name': 'Bogota Marriott Hotel', 'url': 'https://www.marriott.com/BOGMC', 'street': 'Av El Dorado 69 B-53', 'locality': 'Bogota', 'state': '', 'postalcode': '', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/bogmc-bogota-marriott-hotel/?directPageRequest=true', 'coordinate2': '4.659368', 'coordinate3': '-74.108191', 'coordinate4': 'N/A', 'description': "[ Immerse yourself in the culturally rich area surrounding Bogota Marriott Hotel. Located near the area of Sauzalito, our downtown hotel places you near popular destinations in Colombia such as the U.S. Embassy Bogotá, Gran Estactión shopping mall and Simón Bolívar Metropolitan Park. Following an eventful day, find sanctuary in our unique rooms and extended-stay suites combining comfort and convenience. Tune out the city with our soundproof windows while you relax on cotton-rich linens and watch movies on our flat-panel TVs. When you're hungry, stop by Pimento, our casual Italian restaurant, or try Tan

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Marriott,Algiers Marriott Hotel Bab Ezzouar,https://www.marriott.com/ALGMC,"Trust Complex Buildings, Nouveau Quartier des ...",Algiers,,16311,N/A,N/A,N/A,N/A,"[ Discover the Marriott hotel, located in the ...",N/A,N/A,N/A
1,Marriott,Constantine Marriott Hotel,https://www.marriott.com/CZLMC,"Oued Rhumel Street, Cite des Arcades Romaines",Constantine,,,6.616803,N/A,N/A,N/A,[ Marriott Constantine Hotel offers an ideal l...,N/A,4.7,4.6
2,Marriott,Cairo Marriott Hotel & Omar Khayyam Casino,https://www.marriott.com/CAIEG,"16 Saray El Gezira Street, Zamalek",Cairo,,11211,Reviews,https://www.marriott.com/hotels/maps/travel/ca...,30.057162,31.224314,[ Originally constructed as a visitor’s palace...,N/A,4.3,4.2
3,Marriott,"Marriott Mena House, Cairo",https://www.marriott.com/CAIMN,"6 Pyramids Road, Giza",Cairo,,12556,Reviews,https://www.marriott.com/hotels/maps/travel/ca...,29.985647,31.133919,[ Experience the wondrous beauty of the Pyrami...,N/A,4.6,4.3
4,Marriott,Hurghada Marriott Beach Resort,https://www.marriott.com/HRGEG,"El Corniche Road, P.O.Box 38",Hurghada,,12654,Reviews,https://www.marriott.com/hotels/maps/travel/hr...,27.197384,33.844183,"[ Frequented by repeat guests, the Hurghada Ma...",N/A,4.6,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,Marriott,Bogota Marriott Hotel,https://www.marriott.com/BOGMC,Av El Dorado 69 B-53,Bogota,,,https://www.marriott.com/hotels/maps/travel/bo...,4.659368,-74.108191,N/A,[ Immerse yourself in the culturally rich area...,N/A,4.8,4.8
660,Marriott,Cali Marriott Hotel,https://www.marriott.com/CLOMC,Avenida 8 Norte No 10-18,Cali,,,Reviews,https://www.marriott.com/hotels/maps/travel/cl...,3.456097,-76.535922,[ Experience a stay like no other at Cali Marr...,N/A,4.8,4.7
661,Marriott,Medellin Marriott Hotel,https://www.marriott.com/MDEMC,Calle 1a Sur #43a-83,Medellin,,,6.203094,-75.572892,N/A,N/A,[ Transform any trip into a relaxing getaway a...,N/A,4.8,4.5
662,Marriott,Santa Marta Marriott Resort Playa Dormida,https://www.marriott.com/SMRMC,"Carrera 3 No 142-60, Bello Horizonte",Santa Marta,,470006,-74.227578,N/A,N/A,N/A,[ At Santa Marta Marriott Resort Playa Dormida...,N/A,4.8,4.6


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Marriott,Algiers Marriott Hotel Bab Ezzouar,https://www.marriott.com/ALGMC,"Trust Complex Buildings, Nouveau Quartier des ...",Algiers,,16311,N/A,N/A,N/A,N/A,"[ Discover the Marriott hotel, located in the ...",N/A,N/A,N/A
1,Marriott,Constantine Marriott Hotel,https://www.marriott.com/CZLMC,"Oued Rhumel Street, Cite des Arcades Romaines",Constantine,,,6.616803,N/A,N/A,N/A,[ Marriott Constantine Hotel offers an ideal l...,N/A,4.7,4.6
2,Marriott,Cairo Marriott Hotel & Omar Khayyam Casino,https://www.marriott.com/CAIEG,"16 Saray El Gezira Street, Zamalek",Cairo,,11211,Reviews,https://www.marriott.com/hotels/maps/travel/ca...,30.057162,31.224314,[ Originally constructed as a visitor’s palace...,N/A,4.3,4.2
3,Marriott,"Marriott Mena House, Cairo",https://www.marriott.com/CAIMN,"6 Pyramids Road, Giza",Cairo,,12556,Reviews,https://www.marriott.com/hotels/maps/travel/ca...,29.985647,31.133919,[ Experience the wondrous beauty of the Pyrami...,N/A,4.6,4.3
4,Marriott,Hurghada Marriott Beach Resort,https://www.marriott.com/HRGEG,"El Corniche Road, P.O.Box 38",Hurghada,,12654,Reviews,https://www.marriott.com/hotels/maps/travel/hr...,27.197384,33.844183,"[ Frequented by repeat guests, the Hurghada Ma...",N/A,4.6,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,Marriott,Bogota Marriott Hotel,https://www.marriott.com/BOGMC,Av El Dorado 69 B-53,Bogota,,,https://www.marriott.com/hotels/maps/travel/bo...,4.659368,-74.108191,N/A,[ Immerse yourself in the culturally rich area...,N/A,4.8,4.8
660,Marriott,Cali Marriott Hotel,https://www.marriott.com/CLOMC,Avenida 8 Norte No 10-18,Cali,,,Reviews,https://www.marriott.com/hotels/maps/travel/cl...,3.456097,-76.535922,[ Experience a stay like no other at Cali Marr...,N/A,4.8,4.7
661,Marriott,Medellin Marriott Hotel,https://www.marriott.com/MDEMC,Calle 1a Sur #43a-83,Medellin,,,6.203094,-75.572892,N/A,N/A,[ Transform any trip into a relaxing getaway a...,N/A,4.8,4.5
662,Marriott,Santa Marta Marriott Resort Playa Dormida,https://www.marriott.com/SMRMC,"Carrera 3 No 142-60, Bello Horizonte",Santa Marta,,470006,-74.227578,N/A,N/A,N/A,[ At Santa Marta Marriott Resort Playa Dormida...,N/A,4.8,4.6


In [8]:
hotel_df.to_csv('marriott_description.csv')